# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
# Импортируем необходимые библиотеки.
! pip install pandas_profiling
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy import stats as st

In [3]:
# Импорт данных  
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

Изучим данные:

In [ ]:
df_0.head()

In [ ]:
df_1.head()

In [ ]:
df_2.head()

In [ ]:
df_0.info()
df_1.info()
df_2.info()

In [ ]:
# проверим данные на наличие дубликатов в колонке id
df_0['id'].duplicated().sum()

In [ ]:
df_1['id'].duplicated().sum()

In [ ]:
df_2['id'].duplicated().sum()

In [6]:
# Удаляем строки с дупликатами
df_0.drop(df_0.query('id.duplicated() == True').index, inplace=True)
df_0['id'].duplicated().sum()

0

In [7]:
df_1.drop(df_1.query('id.duplicated() == True').index, inplace=True)
df_1['id'].duplicated().sum()

0

In [8]:
df_2.drop(df_2.query('id.duplicated() == True').index, inplace=True)
df_2['id'].duplicated().sum()

0

In [ ]:
ProfileReport(df_0)

In [ ]:
ProfileReport(df_1)

In [ ]:
ProfileReport(df_2)

<div class="alert alert-block alert-warning">
<b>ВЫВОДЫ:</b> 
    
Имеем 3 датасета с данными по трём регионам в отдельности.
-   Чтобы избежать ошибок, в датасетах были удалены строки с дублирующимися id. В результате осталось 99 990 строк, чего достаточно для проведения анализа.
-   Теперь данные готовы для исследования.
-	Для каждого региона были выделены независимые и целевые признаки
-   Изучив тепловые карты для каждого региона, удалось обнаружить высокую корреляцию между признаком f2 и целевым признаком product в регионе "1".
</div>

## Обучение и проверка модели

In [9]:
#создаем модель линейной регрессии.
model = LinearRegression()

In [17]:
# Функция: делим данные на обуч. и валид. выборки, обучим модель на данных регионах,
# делаем предсказания на валидационной выборке, рассчитаем RMSE.
def model_predict(df):
    features = df.drop(['product','id'], axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    predict_mean = predicted.mean()
    rmse = (mean_squared_error(target_valid, predicted))**0.5
    print('Средний запас предсказанного сырья:', predict_mean, 'RMSE:', rmse)
    return target_valid, predicted, predict_mean

In [18]:
# Применим функцию к региону "0"
df_0_target_valid, reg_0_predicted, reg_0_predict_mean = model_predict(df_0)

Средний запас предсказанного сырья: 92.78915638280621 RMSE: 37.853527328872964


In [19]:
# Применим функцию к региону "1"
df_1_target_valid, reg_1_predicted, reg_1_predict_mean = model_predict(df_1)

Средний запас предсказанного сырья: 69.17831957030432 RMSE: 0.892059264771703


In [20]:
# Применим функцию к региону "2"
df_2_target_valid, reg_2_predicted, reg_2_predict_mean = model_predict(df_2)

Средний запас предсказанного сырья: 94.86572480562035 RMSE: 40.07585073246016


**Выводы**

В регионе "0" средний запас предсказанного сырья: 92,8 тыс.баррелей, RMSE = 37,85;

В регионе "1" средний запас предсказанного сырья: 69,2 тыс.баррелей, RMSE = 0,89;

В регионе "2" средний запас предсказанного сырья: 94,9 тыс.баррелей, RMSE = 40,08.

В регионе "2" прогнозируется наибольший запас сырья, но и наибольшая среднеквадратичная ошибка.
В регионе "1" прогнозируется самый низкий из трёх запас сырья, в то время, как среднеквадратичная ошибка очень низкая.

## Подготовка к расчёту прибыли

In [14]:
# Сохраним ключевые значения для расчётов в отдельных переменных:
BEST_COUNT = 200 # Количество лучших точек
BUDGET = 10_000_000_000 # Бюджет на разработку скважин в регионе
INCOME_UNIT = 450_000 # Доход с каждой единицы продукта 

In [16]:
# Рассчитаем достаточный объём сырья для безубыточной разработки одной новой скважины:
volume_raw = (BUDGET / INCOME_UNIT) / BEST_COUNT
round(volume_raw,2)

111.11

In [21]:
# Сравним объём, достаточный для безубыточной разработки, со средним запасом в каждом регионе: 
print('Средний предсказанный запас в регионе "0" меньше нужного объёма для безубыточной разработки на',
      round(volume_raw - reg_0_predict_mean,2))
print('Средний предсказанный запас в регионе "1" меньше нужного объёма для безубыточной разработки на',
      round(volume_raw - reg_1_predict_mean,2))
print('Средний предсказанный запас в регионе "2" меньше нужного объёма для безубыточной разработки на',
      round(volume_raw - reg_2_predict_mean,2))

Средний предсказанный запас в регионе "0" меньше нужного объёма для безубыточной разработки на 18.32
Средний предсказанный запас в регионе "1" меньше нужного объёма для безубыточной разработки на 41.93
Средний предсказанный запас в регионе "2" меньше нужного объёма для безубыточной разработки на 16.25


**Выводы**

После сравнения средних предсказанных моделю запасов сырья в каждом регионе с достаточным объёмом сырья для безубыточной разработки, выяснилось, что среднее значение запасов во всех трех регионах меньше необходимого количества для безубыточной разработки.

Ближе всего к необходимому количеству предсказанные запасы в регионе "2", меньше всего предсказанных запасов в регионе "1".

## Расчёт прибыли и рисков 

**Расчёт прибыли по выбранным скважинам и предсказаниям модели**

In [24]:
# Функция расчёта прибыли по выбранным скважинам и предсказаниям модели:
def profit_calculation(target, predict):
    target = pd.Series(target).reset_index(drop=True)
    predict = pd.Series(predict).reset_index(drop=True)
    predict_max = predict.sort_values(ascending=False)[:BEST_COUNT].index
    return round((target[predict_max].sum() - volume_raw * BEST_COUNT) * INCOME_UNIT / 1000000, 2)

In [25]:

print('Прибыль с 200 лучших скважин региона "0":', profit_calculation(df_0_target_valid, reg_0_predicted), 'млн.руб.')

Прибыль с 200 лучших скважин региона "0": 3365.19 млн.руб.


In [26]:
print('Прибыль с 200 лучших скважин региона "1":', profit_calculation(df_1_target_valid, reg_1_predicted), 'млн.руб.')

Прибыль с 200 лучших скважин региона "1": 2415.09 млн.руб.


In [27]:
print('Прибыль с 200 лучших скважин региона "2":', profit_calculation(df_2_target_valid, reg_2_predicted), 'млн.руб.')

Прибыль с 200 лучших скважин региона "2": 2501.28 млн.руб.


Наибольшая прибыль прогнозируется из региона "0", наименьшая - из региона "1"

**Риски и прибыль для каждого региона**

In [30]:
# Напишем функцию, применяющую технику Bootstrap с 1000 выборок, для получения распределения прибыли.
# Рассчитаем в этой же функции среднюю прибыль,95-й доверительный интервал, риск убытков (отрицательную прибыль).
def bootstrap(target, probabilities):
    state = np.random.RandomState(12345)
    profit = []
    for i in range(1000):
        target_subsample = target.sample(n=500, random_state=state, replace=True)
        probs_subsample = probabilities[target_subsample.index]
        profit.append(profit_calculation(target_subsample, probs_subsample))

    profit = pd.Series(profit)
    loss = (profit < 0).mean()
    mean = profit.mean()
    #confidence_interval = st.t.interval(0.95, df=len(profit)-1, loc=profit.mean(), scale=profit.sem())
    quantile_interval = (profit.quantile(0.025), profit.quantile(0.975))
    
    return round(mean, 2), quantile_interval, round(loss*100, 2)

In [31]:
# прибыль, 95-й доверительный интервал, риск убытков Региона "0"
reg_0_mean, reg_0_interval, reg_0_loss = bootstrap(df_0_target_valid.reset_index(drop=True), reg_0_predicted)
print('Регион "0" средняя прибыль:', reg_0_mean, 'млн.руб.')
print('Регион "0" 95%-й доверительный интервал:', reg_0_interval, 'млн.руб.')
print('Регион "0" Риск убытков:', reg_0_loss, '%')

Регион "0" средняя прибыль: 380.61 млн.руб.
Регион "0" 95%-й доверительный интервал: (-142.94025, 890.9769999999999) млн.руб.
Регион "0" Риск убытков: 7.7 %


In [32]:
# прибыль, 95-й доверительный интервал, риск убытков Региона "1"
reg_1_mean, reg_1_interval, reg_1_loss = bootstrap(df_1_target_valid.reset_index(drop=True), reg_1_predicted)
print('Регион "1" средняя прибыль:', reg_1_mean, 'млн.руб.')
print('Регион "1" 95%-й доверительный интервал:', reg_1_interval, 'млн.руб.')
print('Регион "1" Риск убытков:', reg_1_loss, '%')

Регион "1" средняя прибыль: 478.48 млн.руб.
Регион "1" 95%-й доверительный интервал: (89.52575, 866.6072499999999) млн.руб.
Регион "1" Риск убытков: 1.2 %


In [33]:
# прибыль, 95-й доверительный интервал, риск убытков Региона "2"
reg_2_mean, reg_2_interval, reg_2_loss = bootstrap(df_2_target_valid.reset_index(drop=True), reg_2_predicted)
print('Регион "2" средняя прибыль:', reg_2_mean, 'млн.руб.')
print('Регион "2" 95%-й доверительный интервал:', reg_2_interval, 'млн.руб.')
print('Регион "2" Риск убытков:', reg_2_loss, '%')

Регион "2" средняя прибыль: 330.55 млн.руб.
Регион "2" 95%-й доверительный интервал: (-190.36775, 840.4722499999998) млн.руб.
Регион "2" Риск убытков: 11.3 %


<div class="alert alert-block alert-warning">
**ВЫВОДЫ**

В условиях нашей задачи сказано, что вероятность убытков не должна превышать 2,5%. Данному критерию удовлетворяет только регион "1". К тому же данный регион имеет наилучший прогноз по средней прибыли,<b> нет отрицательных значений в доверительном интервале прибыли.</b>

Наиболее перспективным будет разработка месторождений в регионе 1.
</div>